In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
from collections import OrderedDict
from ontology import *

# Load brain and text data

In [2]:
act_bin = load_coordinates().astype(float)
print("Document N={}, Structure N={}".format(act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=114


In [3]:
version = 190325
dtm_bin = load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=4107


In [4]:
frameworks = ["data-driven", "rdoc", "dsm"]
suffixes = ["", "_opsim", "_opsim"]
lists, circuits = {}, {}
for fw, suffix in zip(frameworks, suffixes):
    lists[fw], circuits[fw] = load_framework(fw, suffix=suffix)

In [5]:
scores = {fw: score_lists(lists[fw], dtm_bin, label_var="DOMAIN") 
          for fw in frameworks}

In [6]:
words = []
for fw in frameworks:
    words += list(lists[fw]["TOKEN"])
words = sorted(list(set(words)))
structures = list(act_bin.columns)
print("Term N={}, Structure N={}".format(len(words), len(structures)))

Term N=297, Structure N=114


In [7]:
domains = {fw: list(OrderedDict.fromkeys(lists[fw]["DOMAIN"])) for fw in frameworks}

In [8]:
pmids = act_bin.index.intersection(scores["rdoc"].index).intersection(scores["dsm"].index)
len(pmids)

18155

In [9]:
for fw in frameworks:
    scores[fw] = scores[fw].loc[pmids]

In [10]:
dtm_bin = dtm_bin.loc[pmids, words]
act_bin = act_bin.loc[pmids, structures]

# Load frameworks

In [11]:
systems = {}
for fw in frameworks:
    fw_df = pd.DataFrame(0.0, index=words+structures, columns=domains[fw])
    for dom in domains[fw]:
        for word in lists[fw].loc[lists[fw]["DOMAIN"] == dom, "TOKEN"]:
            fw_df.loc[word, dom] = 1.0
        for struct in structures:
            fw_df.loc[struct, dom] = circuits[fw].loc[struct, dom]
    fw_df[fw_df > 0.0] = 1.0
    systems[fw] = fw_df

# Similarity of RDoC and data-driven systems

## Observed values

In [12]:
from scipy.spatial.distance import dice, cdist

In [13]:
def compute_sim_obs(fw):
    sims = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            sims.loc[k,r] = 1.0 - dice(systems["data-driven"][k], systems[fw][r])
    return sims

In [14]:
sims = compute_sim_obs("rdoc")
sims

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0.206897,0.0481928,0,0.21978,0.240964,0
REWARD,0.219512,0.487179,0,0.0232558,0.0512821,0
COGNITION,0.372549,0.408163,0.0380952,0.0754717,0.22449,0.02
VISION,0,0,0.264151,0.168224,0,0.0990099
MANIPULATION,0,0,0.471698,0,0,0.594059
MEANING,0.134831,0,0,0.215054,0.0941176,0
LANGUAGE,0.0842105,0,0,0.10101,0.131868,0.129032


## Null distribution

In [15]:
def compute_sim_null(fw, n_iter=10000):
    sims_null = np.empty((len(domains["data-driven"]), len(domains[fw]), n_iter))
    for n in range(n_iter):
        null = np.random.choice(words+structures, 
                                size=len(words+structures), replace=False)
        sims_null[:,:,n] = 1.0 - cdist(systems["data-driven"].loc[null].values.T, 
                                       systems[fw].values.T, metric="dice")
        if n % (float(n_iter) / 10.0) == 0:
            print("Iteration {}".format(n))
    return sims_null

In [16]:
sims_null = compute_sim_null("rdoc")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [17]:
from statsmodels.stats.multitest import multipletests

In [18]:
def compute_sim_fdr(fw, sims, sims_null):
    n_iter = sims_null.shape[2]
    pvals = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for i, k in enumerate(domains["data-driven"]):
        for j, r in enumerate(domains[fw]):
            pvals.loc[k,r] = np.sum(sims_null[i,j,:] > sims.loc[k,r]) / float(n_iter)
    fdrs = multipletests(pvals.values.ravel(), method="fdr_bh")[1]
    fdrs = np.reshape(fdrs, pvals.shape)
    fdrs = pd.DataFrame(fdrs, index=domains["data-driven"], columns=domains[fw])
    return fdrs

In [19]:
fdrs = compute_sim_fdr("rdoc", sims, sims_null)
fdrs

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0.01134,0.9996,0.9996,0.00746667,0.0028,0.9996
REWARD,0.004725,0,0.9996,0.9996,0.9996,0.9996
COGNITION,0,0,0.9996,0.9996,0.02415,0.9996
VISION,0.9996,0.9996,0.003,0.4146,0.9996,0.9996
MANIPULATION,0.9996,0.9996,0,0.9996,0.9996,0
MEANING,0.4146,0.9996,0.9996,0.0133636,0.974153,0.9996
LANGUAGE,0.9996,0.9996,0.9996,0.9996,0.54768,0.621862


In [20]:
def compute_sim_star(fw, fdrs):
    stars = pd.DataFrame("", index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            fdr = fdrs.loc[k,r]
            if fdr < 0.05:
                stars.loc[k,r] = "*"
            if fdr < 0.01:
                stars.loc[k,r] = "**"
            if fdr < 0.001:
                stars.loc[k,r] = "***"
    return stars

In [21]:
stars = compute_sim_star("rdoc", fdrs)
stars

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,*,,,**,**,
REWARD,**,***,,,,
COGNITION,***,***,,,*,
VISION,,,**,,,
MANIPULATION,,,***,,,***
MEANING,,,,*,,
LANGUAGE,,,,,,


## Weights for figure

In [22]:
sims[fdrs < 0.05] * 40

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,8.27586,NaN,NaN,8.79121,9.63855,NaN
REWARD,8.78049,19.4872,NaN,NaN,NaN,NaN
COGNITION,14.902,16.3265,NaN,NaN,8.97959,NaN
VISION,NaN,NaN,10.566,NaN,NaN,NaN
MANIPULATION,NaN,NaN,18.8679,NaN,NaN,23.7624
MEANING,NaN,NaN,NaN,8.60215,NaN,NaN
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN


# Similarity of DSM and data-driven systems

## Observed values

In [23]:
sims = compute_sim_obs("dsm")
sims

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0,0,0.0909091,0.08,0.27451,0,0.311111,0,0
REWARD,0,0,0.0512821,0,0,0.157895,0,0.292683,0.2
COGNITION,0,0,0.101695,0.0923077,0.121212,0.0689655,0.0666667,0.0655738,0
VISION,0.273684,0,0.0333333,0,0,0,0,0,0
MANIPULATION,0.231579,0.246154,0,0,0,0,0,0,0.0327869
MEANING,0.0740741,0.0392157,0,0,0,0,0.0425532,0,0
LANGUAGE,0.137931,0.0701754,0,0,0,0.0392157,0,0,0


## Null distribution

In [24]:
sims_null = compute_sim_null("dsm")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [25]:
fdrs = compute_sim_fdr("dsm", sims, sims_null)
fdrs

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0.9965,0.967995,0.416373,0.72282,0,0.967995,0,0.967995,0.967995
REWARD,0.98941,0.967995,0.793429,0.967995,0.967995,0.0496125,0.967995,0,0.0084
COGNITION,0.9965,0.967995,0.3661,0.72282,0.4095,0.72282,0.793429,0.832595,0.967995
VISION,0.00126,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995
MANIPULATION,0.0324,0.00126,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995
MEANING,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995
LANGUAGE,0.72282,0.832595,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995,0.967995


In [26]:
stars = compute_sim_star("dsm", fdrs)
stars

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,,,,,***,,***,,
REWARD,,,,,,*,,***,**
COGNITION,,,,,,,,,
VISION,**,,,,,,,,
MANIPULATION,*,**,,,,,,,
MEANING,,,,,,,,,
LANGUAGE,,,,,,,,,


## Weights for figure

In [27]:
sims[fdrs < 0.05] * 40

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,NaN,NaN,NaN,NaN,10.9804,NaN,12.4444,NaN,NaN
REWARD,NaN,NaN,NaN,NaN,NaN,6.31579,NaN,11.7073,8
COGNITION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VISION,10.9474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANIPULATION,9.26316,9.84615,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MEANING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LANGUAGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
